# Baseline Experiment - Pre-optimized Solution

This notebook loads the best pre-optimized solution from snapshots and verifies its score.

In [1]:
import pandas as pd
import numpy as np
import math
from numba import njit
import os

# Tree geometry
TX = np.array([0, 0.125, 0.0625, 0.2, 0.1, 0.35, 0.075, 0.075, -0.075, -0.075, -0.35, -0.1, -0.2, -0.0625, -0.125])
TY = np.array([0.8, 0.5, 0.5, 0.25, 0.25, 0, 0, -0.2, -0.2, 0, 0, 0.25, 0.25, 0.5, 0.5])

print(f"Tree has {len(TX)} vertices")
print(f"Tree width: {max(TX) - min(TX):.3f}")
print(f"Tree height: {max(TY) - min(TY):.3f}")

Tree has 15 vertices
Tree width: 0.700
Tree height: 1.000


In [2]:
@njit
def score_group(xs, ys, degs, tx, ty):
    """Calculate score for a single N-tree configuration"""
    n = xs.size
    V = tx.size
    mnx = mny = 1e300
    mxx = mxy = -1e300
    for i in range(n):
        r = degs[i] * math.pi / 180.0
        c, s = math.cos(r), math.sin(r)
        for j in range(V):
            X = c * tx[j] - s * ty[j] + xs[i]
            Y = s * tx[j] + c * ty[j] + ys[i]
            mnx, mxx = min(mnx, X), max(mxx, X)
            mny, mxy = min(mny, Y), max(mxy, Y)
    side = max(mxx - mnx, mxy - mny)
    return side * side / n

def calculate_total_score(df):
    """Calculate total score for a submission dataframe"""
    total_score = 0.0
    scores_by_n = {}
    
    for n in range(1, 201):
        # Get rows for this N
        mask = df['id'].str.startswith(f'{n:03d}_')
        group = df[mask]
        
        if len(group) != n:
            print(f"Warning: N={n} has {len(group)} trees, expected {n}")
            continue
        
        # Parse coordinates (remove 's' prefix)
        xs = group['x'].str[1:].astype(float).values
        ys = group['y'].str[1:].astype(float).values
        degs = group['deg'].str[1:].astype(float).values
        
        # Calculate score
        score = score_group(xs, ys, degs, TX, TY)
        scores_by_n[n] = score
        total_score += score
    
    return total_score, scores_by_n

print("Scoring functions defined")

Scoring functions defined


In [3]:
# Load the pre-optimized submission
submission_path = '/home/nonroot/snapshots/santa-2025/21328309254/code/submission.csv'
df = pd.read_csv(submission_path)
print(f"Loaded submission with {len(df)} rows")
print(df.head())

Loaded submission with 20100 rows
      id                    x                    y                   deg
0  001_0   s0.011947114128223  s-0.262123336599447   s44.999999999999744
1  002_0   s0.154097069621364  s-0.038540742694785  s203.629377730656842
2  002_1  s-0.154097069621364  s-0.561459257305215   s23.629377730656792
3  003_0   s0.185770648104683  s-0.547448842335598  s111.125132292892999
4  003_1   s0.296170527806543  s-0.052551157664402   s66.370622269343002


In [4]:
# Calculate total score
total_score, scores_by_n = calculate_total_score(df)
print(f"\nTotal Score: {total_score:.6f}")
print(f"Expected: 70.647327")
print(f"Difference: {abs(total_score - 70.647327):.6f}")


Total Score: 70.647327
Expected: 70.647327
Difference: 0.000000


In [5]:
# Analyze score contribution by N
print("\nTop 20 N values by score contribution:")
sorted_scores = sorted(scores_by_n.items(), key=lambda x: x[1], reverse=True)
for n, score in sorted_scores[:20]:
    print(f"  N={n:3d}: {score:.6f}")


Top 20 N values by score contribution:
  N=  1: 0.661250
  N=  2: 0.450779
  N=  3: 0.434745
  N=  5: 0.416850
  N=  4: 0.416545
  N=  7: 0.399897
  N=  6: 0.399610
  N=  9: 0.387415
  N=  8: 0.385407
  N= 15: 0.379203
  N= 10: 0.376630
  N= 21: 0.376451
  N= 20: 0.376057
  N= 22: 0.375258
  N= 11: 0.374924
  N= 16: 0.374128
  N= 26: 0.373997
  N= 12: 0.372724
  N= 13: 0.372294
  N= 25: 0.372144


In [6]:
# Calculate packing efficiency for each N
print("\nPacking efficiency analysis:")
tree_area = 0.3325  # Approximate area of tree polygon

for n in [1, 2, 5, 10, 20, 50, 100, 200]:
    score = scores_by_n[n]
    side = math.sqrt(score * n)
    box_area = side * side
    total_tree_area = n * tree_area
    efficiency = total_tree_area / box_area * 100
    print(f"  N={n:3d}: side={side:.4f}, efficiency={efficiency:.1f}%")


Packing efficiency analysis:
  N=  1: side=0.8132, efficiency=50.3%
  N=  2: side=0.9495, efficiency=73.8%
  N=  5: side=1.4437, efficiency=79.8%
  N= 10: side=1.9407, efficiency=88.3%
  N= 20: side=2.7425, efficiency=88.4%
  N= 50: side=4.2471, efficiency=92.2%
  N=100: side=5.8782, efficiency=96.2%
  N=200: side=8.2164, efficiency=98.5%


In [7]:
# Copy to submission folder
import shutil
os.makedirs('/home/submission', exist_ok=True)
shutil.copy(submission_path, '/home/submission/submission.csv')
print("Submission copied to /home/submission/submission.csv")

# Verify the copy
df_verify = pd.read_csv('/home/submission/submission.csv')
print(f"Verified: {len(df_verify)} rows")

Submission copied to /home/submission/submission.csv
Verified: 20100 rows


In [8]:
# Save metrics
import json
metrics = {
    'cv_score': total_score,
    'scores_by_n': {str(k): v for k, v in scores_by_n.items()}
}

with open('/home/code/experiments/001_baseline/metrics.json', 'w') as f:
    json.dump(metrics, f, indent=2)

print(f"Metrics saved. CV Score: {total_score:.6f}")

Metrics saved. CV Score: 70.647327
